In [1]:
import socket
def read_paramters():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 12345)
    client_socket.connect(server_address)

    # Receive data from the server
    data = client_socket.recv(1024)
    received_integer = int(data.decode('utf-8'))
    print('Received integer:', received_integer)
    client_socket.close()
    return received_integer
def send_public_key(public_key):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost',1111)
    server_socket.bind(server_address)
    server_socket.listen(1)

    print('Waiting for a connection...')

    client_socket, client_address = server_socket.accept()
    print('Connected by', client_address)

    # Send an integer to the client
    
    client_socket.sendall(str(public_key).encode('utf-8'))
    client_socket.close()
    server_socket.close()
    

In [2]:
p=read_paramters()

Received integer: 24181541446232922176359640334881515440582115395176030149783119793900422742015207110170179019485447177003051104748171105624051375882375080105315909391300108532498040565919047449742837234262613225671308907903078289404729925522625524359794183441459810613114997607671207343289153032600106332149163455867800475798697000172160174420354071206027651384402679666698032519546422159294381966241274915580089630883852507178584043395529780029124678815203328325080892221876535929418008962485138236592458707109806203111278934562939363762867828897915415728243061435880191782935166990369482069854113805604820875528544526870545841242459


In [3]:
g=read_paramters()

Received integer: 9127469538228324700865522973167654700200922912762281908149888090931524742022721808148997888153486498990104584132575851208333617120115605119330578385156732386068380640035921372255701104243473053296349940246085498903524590632716596136447598401728673894377330730218626796052665878711442848772966024454916385183667356544353756253105065070714736080169543321113355380543255655828782036032622037498270205462813878258589378800303427897799009803117178185318329394217867245391345142858627930704734802904896740535488099022606255796492995529056968326669080398147147669736964531081465707976178171270628839414790830796330501487275


In [4]:
from random import randint

Alice_private_key = randint(2, p - 2)
print("Alice_Private_key:\n",Alice_private_key)

Alice_Private_key:
 4990488296262106308425889974504778735739222706597079885077035600038662811194058297710043215866031010996625469401538940447004931526366626240349599880096305641548626279728920797420947258511694317643591136812319853655161886560142770793046336346932601644670623795366994107869611346226907454150044847646873231583748467648899509793876932350870687966679936243685536007733178089486487612131265555053477798742324700068848733273731501979841158191211442899788290410327047228910719177467175760930818746247966840854218889131036647655171366039525801602568194938520465749162886611416841200006424541578258211353301283124623866619654


In [5]:
def mod_exp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent //= 2
        base = (base * base) % modulus
    return result

In [6]:
Alice_public_key = mod_exp(g, Alice_private_key, p)
print("Alice_Public_key:\n",Alice_public_key)

Alice_Public_key:
 7129299296205506582106390138532099103738059394779090673931635624853055737933529134852343042931110849691578081400772307541188106543861531361066057363268964163961856907201609159852933802887939546796275142955655800291999368957889672284327496300130049997876429440264766799441229944880255481186975048028909229179335586394091809985414120596359397309200738127576385968398536482653836193017462716150281326093735025175430195687561737333567576282882101496751497737634411055528874652251007376530751136774677645878617342927927064613816639900284373320715455316596220056595767914947397801425951483282714096277225778959887320682132


In [8]:
send_public_key(Alice_public_key)

Waiting for a connection...
Connected by ('127.0.0.1', 55280)


In [7]:
bob_public_key=read_paramters()

Received integer: 2307318120642966431663895199235877783225365729269979662677006727111535525222854121367449550341174836187579056507538792328714007508239526482436191494275110813615815332583991811063098202105034970522270453920300456717388297352062295479014502586860736353310756785886272344052878535340277689769200867391101408830843600993647472708168576212987824871926232147215901213493643187656388330901838317333956730119421081956324006695445649829440322069918931866372561238031833759876524703955313813586775554922568795895346391982007127814923848568476865091698260592587113306538534118837637829286592397006111814798056892865727531998554


In [9]:
common_key_Alice = mod_exp(bob_public_key, Alice_private_key, p)
print("The common key calculated by Alice:\n",common_key_Alice)

The common key calculated by Alice:
 5272430107689889097796882842610122745648554468770498663219568113872026188333003156976891029175501913106236541955306563766350079827396832650628506723788137581421519737847006894122314462787409524471745092924827744272669737221005493460648647127975978015324848224050557401217996594479640782598114631839962397856336983935782017444628415724269679739912528520160499286677668549067205662483072170790787336480267053492579547329706096943112809192329777259952470722285790890709063335185949459694744488691851225665518326894656357685888721847145326591059889122456731226181735870141684045998786988139018748063668157906261608528929


In [12]:
#convert common key to bytes
common_key_Alice_bytes=common_key_Alice.to_bytes(10000, byteorder='big')


In [13]:
import hashlib

def derive_key(original_key, output_length=16):
    # Use SHA-256 hash function
    sha256 = hashlib.sha256()

    # Update the hash object with the original key
    sha256.update(original_key)

    # Get the digest (hash) and truncate it to the desired length
    derived_key = sha256.digest()[:output_length]

    return derived_key

In [14]:
#get 16 bytes from common key using hash function

derive_key_Alice=derive_key(common_key_Alice_bytes)
derive_key_Alice

b'\x04v5\xc1~G\x16B\x89\x90E`p\xf6\xa0v'

In [16]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from pydub import AudioSegment
import os

# Set the path to the ffmpeg executable
AudioSegment.converter = "ffmpeg.exe"

def encrypt_audio(input_audio_path, output_audio_path, key, commun_key):
    # Load the audio file
    
    audio = AudioSegment.from_file(input_audio_path)

    # Convert audio to raw data (bytearray)
    audio_data = audio.raw_data

    # IV (Initialization Vector) from commun key 
    iv = commun_key[:16]

    # Pad the data using PKCS7 padding
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(audio_data) + padder.finalize()

    # Create an AES cipher object with the key, mode, and IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the padded data
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted audio to a new file
    encrypted_audio = AudioSegment(encrypted_data, frame_rate=audio.frame_rate, sample_width=audio.sample_width, channels=audio.channels)
    encrypted_audio.export(output_audio_path, format="wav")


In [17]:
import pyaudio
import wave
import pygame


def record_audio(file_path, duration=5, sample_rate=44100, channels=1, chunk_size=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=chunk_size)

    print("Recording...")

    frames = []

    for i in range(0, int(sample_rate / chunk_size * duration)):
        data = stream.read(chunk_size)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

def play_audio(file_path):
    pygame.init()
    pygame.mixer.init()

    try:
        # Load the audio file
        pygame.mixer.music.load(file_path)

        # Play the audio file
        pygame.mixer.music.play()

        # Wait for the audio to finish playing
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    except pygame.error as e:
        print(f"Error playing audio: {e}")
    finally:
        # Cleanup
        pygame.mixer.quit()
        pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [18]:
output_file = "recorded_audio.wav"
record_duration = 5  # seconds

record_audio(output_file, duration=record_duration)
print(f"Audio recorded and saved to {output_file}")

Recording...
Recording finished.
Audio recorded and saved to recorded_audio.wav


In [19]:
#play record_audio
play_audio("recorded_audio.wav")

In [20]:
# input audio:

input_audio_path = 'recorded_audio.wav'
output_audio_path = 'encrypted_audio.wav'

#Alice encrypt input audio by 
encrypt_audio(input_audio_path, output_audio_path, derive_key_Alice)

In [21]:
play_audio("encrypted_audio.wav")